### Imported libraries and Scripts

In [2]:
### imported Libraries
import pandas as pd
import numpy as np
import importlib
import sqlite3
import datetime
from pprint import pprint
from sklearn.cluster import KMeans

### Sklearn imported libraries
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.cluster import KMeans
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.preprocessing import MinMaxScaler

# Pipeline for Oversampling
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE

from sklearn import metrics
import matplotlib.pyplot as plt

### Imported Scripts
import utils

%load_ext autoreload
%autoreload 2

importlib.reload(utils)

<module 'utils' from 'd:\\FEUP\\4 Ano\\AC\\ac-feup\\jupyters\\utils.py'>

### Choose Macros

In [3]:
OVERSMAPLE = True
DEBUG = True
WRITE = True
DUMMIES = True
CATEGORY_ENCONDING = False
MIN_MAX_SCALER = False
SPLIT_RATIO = 0.8
N_COLUMNS = 15
N_SPLITS = 3

## Defined Methods

### Choose Dataset Options

In [4]:
def add_dummy(df,columns):
    copy= df.copy()

    for column in columns:
        dummies = pd.get_dummies(copy[column])
        copy = copy.drop(column,axis=1)
        copy = copy.join(dummies)
    
    return copy

def convert_dates(df):
    copy = df.copy()
    columns = ["loan_date","account_creation","birth_number"]

    for column in columns:
        copy[column] = copy[column].apply(lambda x: datetime.datetime.strptime(x, '%d-%m-%Y').strftime('%Y')).astype(int)

    copy["age_on_loan"] = copy["loan_date"] - copy["birth_number"]
    copy = copy.drop(columns = ["loan_date","account_creation","birth_number"])

    copy['card_issued'] = pd.to_numeric(copy["card_issued"].astype(str), errors='coerce').fillna(1).astype(int)

    return copy

def get_df(test=False):
    con = sqlite3.connect("../database/banking_data")
    if test:
        df = pd.read_sql_query("SELECT * FROM loan_united_test", con)
    else:
        df = pd.read_sql_query("SELECT * FROM loan_united_train", con)

    df = convert_dates(df)
    con.close()

    if DUMMIES:
        columns = ["account_frequency","gender","card_type"]
        df = add_dummy(df, columns)

    if CATEGORY_ENCONDING:
        df = utils.normalize_category(df)

    
    if MIN_MAX_SCALER:
        scaler = MinMaxScaler()
        copy = df.copy()
        y = copy["status"]
        X = copy.drop(columns=["status"])
        transf = scaler.fit_transform(X)
        copy = pd.DataFrame(transf,index=X.index,columns=X.columns)
        copy["status"] = y
        df = copy
        
    
    return df


### Data splitting

In [5]:
### Split the data
def split_dataset(df):

    ### Seperate the precition columns from output
    
    X = df.drop(columns=['status'])
    y = df['status']

    select = SelectKBest(f_classif, k= N_COLUMNS)
    X_new = select.fit_transform(X, y)

    split_filter = select.get_support()
    features = X.columns

    ### Apply splitting
    X_train, X_test, y_train, y_test = train_test_split(X_new,y,train_size=SPLIT_RATIO,test_size=1-SPLIT_RATIO)

    return X_train,X_test,y_train,y_test, features[split_filter]

### Algorithms

In [6]:
def get_random_forest():
    return RandomForestClassifier(bootstrap = False,
                                    max_depth =110,
                                    max_features = 2,
                                    min_samples_leaf = 3,
                                    min_samples_split = 8,
                                    n_estimators = 300)

In [7]:
def get_logistic_regression():
    return LogisticRegression(random_state=10,
                                solver='lbfgs',
                                max_iter=200)

In [8]:
def get_decision_tree():
    return DecisionTreeClassifier(criterion='entropy',
                                    max_depth=5,
                                    max_features="sqrt",
                                    min_impurity_split="0.1",
                                    min_samples_leaf=4,
                                    min_samples_split=8,
                                    splitter='random')

In [9]:
def get_knn():
    return KNeighborsClassifier(algorithm="ball_tree",
                                    leaf_size="50",
                                    metric="chebyshev",
                                    n_neighbors=10,
                                    p=10,
                                    weights="uniform")

### Use *Grid Search Cross Validation* to find the best grid for an algorithm

In [10]:
def create_rf():
    return  RandomForestClassifier()
def create_knn():
    return  KNeighborsClassifier()
def create_dt():
    return  DecisionTreeClassifier()
def create_lr():
    return  LogisticRegression()

In [11]:
### Uses a grid search to generate random parameters to find the best grid model
def getBestSearch(algorithm,grid):
    train = get_df()

    X = train.drop(columns=['status'])
    y = train['status']

    if algorithm == "RF":
        alg = build_pipeline(create_rf())
    elif algorithm == "KNN":
        alg = build_pipeline(create_knn())
    elif algorithm == "DT":
        alg = build_pipeline(create_dt())
    elif algorithm == "LR":
        alg = build_pipeline(create_lr())
    
    grid_search = GridSearchCV(estimator = alg,
                               param_grid = grid, 
                               scoring=metrics.make_scorer(utils.get_auc, greater_is_better=True),
                               cv=StratifiedKFold(2,random_state=30,shuffle=True),
                               n_jobs = -1,
                               verbose = 2)

    model = grid_search.fit(X,y)

    if DEBUG:
        print('Best Score: ', model.best_score_)
        print('Best Params: ', model.best_params_)
    
    return model.best_score_, model.best_params_


### Build a Pipeline to apply a sampling and a classification algorithm

In [12]:
### TODO: Add undersample before final delivery
def build_pipeline(algorithm):

    if(OVERSMAPLE):
        return Pipeline([
            ('sampling',SMOTE()),
            ('classification',algorithm)
        ])
    else:
        return  Pipeline([
            ('classification',algorithm)
        ])

### Final Stratified Cross Validation

In [13]:
def final_CV(algorithm):
    train = get_df()
    # train = utils.normalization(train,'status')

    X = train.drop(columns=['status'])
    y = train['status']

    select = SelectKBest(f_classif, k=N_COLUMNS)
    X_new = select.fit_transform(X, y)

    split_filter = select.get_support()
    features = X.columns[split_filter]

    skf = StratifiedKFold(n_splits=N_SPLITS, random_state=True, shuffle=True)

    model_list = []
    auc_list = []
    
    for train_index, test_index in skf.split(X_new, y):

        X_train, X_test = X_new[train_index], X_new[test_index]
        y_train, y_test = y[train_index], y[test_index]

        ### Train the model
        pipe = build_pipeline(algorithm())
        model = pipe.fit(X_train, y_train)

        ### Predict the outcome with the test data
        y_pred = model.predict_proba(X_test)
        y_final = y_pred.transpose()[0]
        
        auc = utils.get_auc(y_test, y_final)
        auc_list.append(auc)
        model_list.append(model)
        print(f"AUC={auc}")
        
    ### Get the best model

    best_score = max(auc_list)
    best_model = model_list[auc_list.index(best_score)]
    
    
    ### Use the best model to get a prediction
    test = get_df(test=True)
    
    X2 = test.drop(columns=['status'])
    X2 = X2[features]

    y_predicted = best_model.predict_proba(X2)
    y_final = y_predicted.transpose()[0]
    
    final_df = pd.DataFrame()
    final_df['Id'] = test["loan_id"]
    final_df['Predicted'] = y_final

    avg = sum(auc_list)/len(auc_list)
    print(f"Average AUC = {avg}")
    print(f"Best AUC = {best_score}")
    
    
    if DEBUG:
        print(f"Predictions:\n {final_df}")
    
    if WRITE:
        final_df.to_csv('../csvs/results/final.csv', index=False)
        print("Sucessfully stored the predictions in a file named 'final.csv'")

    
    


### Running an algorithm

In [14]:
def training_algorithm(algorithm):
    ### Getting the dataset
    train = get_df(test=False)
    ### Getting a Model from training
    
    X_train,X_test,y_train,y_test, features = split_dataset(train)

    
    pipe = build_pipeline(algorithm())
    model = pipe.fit(X_train,y_train)

    y_predicted = model.predict_proba(X_test)

    y_final = y_predicted.transpose()[0]

    if DEBUG:
        score = model.score(X_test,y_test)
        auc = utils.get_auc(y_test,y_final,label=-1)
        # plot_auc(y_test,y_final)
        # conf_matrix(model,y_test,y_final)
        print(f"Score: {score}")
        print(f"Auc: {auc}")
    
    return model, features

In [15]:
def testing_model(model,features):

    test = get_df(test=True)

    X = test.drop(columns=['status'])
    X = X[features]

    print(X.columns)

    y_predicted = model.predict_proba(X)
    y_final = y_predicted.transpose()[0]
    
    final_df = pd.DataFrame()
    final_df['Id'] = test["loan_id"]
    final_df['Predicted'] = y_final

    if DEBUG:
        print(f"Predictions:\n {final_df}")
        
    
    if WRITE:
        final_df.to_csv('../csvs/results/testing_model.csv', index=False)
        print("Sucessfully stored the predictions in a file named 'testing_model.csv'")

In [16]:
def run_algorithm(algorithm):
    
    if(DEBUG):
        print("Running the provided algorithm")
    
    model, features = training_algorithm(algorithm)
    testing_model(model,features)

## Running the algorithms

### Choose your algorithm:

In [22]:
### Leave the one you want to run uncommented
# algorithm = get_random_forest
# algorithm = get_logistic_regression
# algorithm = get_decision_tree
algorithm = get_knn

### Run with train_test_split

In [23]:
run_algorithm(algorithm)

Running the provided algorithm


TypeError: '<' not supported between instances of 'str' and 'int'

### Run with StratifiedKFold

In [19]:
final_CV(algorithm) 

AUC=0.6283244680851063
AUC=0.8368794326241136
AUC=0.7226950354609929
Average AUC = 0.729299645390071
Best AUC = 0.8368794326241136
Predictions:
        Id  Predicted
0    5895   0.124218
1    7122   0.623908
2    6173   0.243186
3    6142   0.380753
4    5358   0.419930
..    ...        ...
349  4989   0.227963
350  5221   0.230826
351  6402   0.194312
352  5346   0.222404
353  6748   0.239195

[354 rows x 2 columns]
Sucessfully stored the predictions in a file named 'final.csv'


C:\Users\Tester\anaconda3\lib\site-packages\sklearn\base.py:438: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


### Finding the best grid

#### Decision Tree

In [20]:
param_grid = {
    'classification__criterion':['gini', 'entropy'],
    'classification__splitter': ["best", "random"],
    'classification__max_depth': [5, 10, 20, 30, 40],
    'classification__min_samples_split': [2, 4, 6, 8],
    'classification__min_samples_leaf': [1, 2, 4, 6],
    'classification__max_features': ["auto", "sqrt", "log2"],
    'classification__min_impurity_decrease': [0.05, 0.1, 0.2, 0.3],
    'classification__class_weight': ["balanced", None]
}

getBestSearch("DT",param_grid)

Fitting 2 folds for each of 7680 candidates, totalling 15360 fits


KeyboardInterrupt: 

#### Random Forest

In [ ]:
param_grid = {
    'classification__max_depth': [80, 90, 100, 110],
    'classification__max_features': [2, 3],
    'classification__min_samples_leaf': [3, 4, 5],
    'classification__min_samples_split': [8, 10, 12],
    'classification__n_estimators': [100, 200, 300, 1000]
}

getBestSearch("RF",param_grid)

Fitting 2 folds for each of 288 candidates, totalling 576 fits
Best Score:  0.5121800801726797
Best Params:  {'classification__max_depth': 110, 'classification__max_features': 2, 'classification__min_samples_leaf': 3, 'classification__min_samples_split': 8, 'classification__n_estimators': 300}


(0.5121800801726797,
 {'classification__max_depth': 110,
  'classification__max_features': 2,
  'classification__min_samples_leaf': 3,
  'classification__min_samples_split': 8,
  'classification__n_estimators': 300})

#### Logistic Regression

In [24]:
param_grid = {}
 
getBestSearch("LR",param_grid)

Fitting 2 folds for each of 1 candidates, totalling 2 fits
Best Score:  0.3667900092506938
Best Params:  {}


C:\Users\Tester\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


(0.3667900092506938, {})

#### K-Nearest Neighbours

In [25]:
param_grid = {
    'classification__n_neighbors': [2, 3, 4, 5, 8, 10],
    'classification__algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
    'classification__weights': ['uniform', 'distance'],
    'classification__metric': ['euclidean', 'manhattan', 'chebyshev', 
        'minkowski', 'wminkowski', 'seuclidean', 'mahalanobis', 'haversine', 
        'hamming', 'canberra', 'braycurtis', 'cityblock', 'infinity', 
        'l1', 'l2', 'p'],
    'classification__leaf_size': [10, 20, 30, 50, 80, 150],
    'classification__p': [1, 2, 3, 5, 10]
}

getBestSearch("KNN",param_grid)

Fitting 2 folds for each of 23040 candidates, totalling 46080 fits


C:\Users\Tester\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
9360 fits failed out of a total of 46080.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
720 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Tester\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 681, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Tester\anaconda3\lib\site-packages\imblearn\pipeline.py", line 266, in fit
    self._final_estimator.fit(Xt, yt, **fit_params_last_step)
  File "C:\Users\Tester\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py", line 198, in fit
    return self._fit(X, y)
  File "C:\Us

Best Score:  0.5868023435090965
Best Params:  {'classification__algorithm': 'ball_tree', 'classification__leaf_size': 150, 'classification__metric': 'chebyshev', 'classification__n_neighbors': 10, 'classification__p': 3, 'classification__weights': 'distance'}


C:\Users\Tester\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:969: UserWarning: One or more of the test scores are non-finite: [0.49429541 0.50300648 0.52489978 ...        nan        nan        nan]
  warnings.warn(


(0.5868023435090965,
 {'classification__algorithm': 'ball_tree',
  'classification__leaf_size': 150,
  'classification__metric': 'chebyshev',
  'classification__n_neighbors': 10,
  'classification__p': 3,
  'classification__weights': 'distance'})

In [ ]:
def plot_auc(y_test,y_predicted):
    
    fpr, tpr, _ = metrics.roc_curve(y_test, y_predicted,pos_label=-1)
    roc_auc = metrics.auc(fpr, tpr)
    
    plt.title('Receiver Operating Characteristic')
    plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
    plt.fill_between(fpr,tpr,color="lightskyblue")
    plt.legend(loc = 'lower right')
    plt.plot([0, 1], [0, 1],'w--')
    plt.xlim([0, 1])
    plt.ylim([0, 1.01])
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.show()

### Clustering

In [ ]:
con = sqlite3.connect("../database/banking_data")

train = pd.read_sql_query("SELECT * FROM loan_united_train", con)

con.close()

df = utils.normalize_category(train)
# df = utils.normalization(df,'status')

X = df.drop(columns=['status'])

kmeans_model = KMeans(n_clusters=3).fit(X)
# labels = kmeans_model.labels_
# metrics.silhouette_score(X, labels, metric='euclidean')

pprint(kmeans_model)    


KMeans(n_clusters=3)


In [ ]:
get_df().columns

Index(['index', 'loan_id', 'loan_amount', 'loan_duration', 'payments',
       'status', 'no. of inhabitants',
       'account_district no. of municipalities with inhabitants < 499',
       'account_district no. of municipalities with inhabitants 500-1999',
       'account_district no. of municipalities with inhabitants 2000-9999',
       'account_district no. of municipalities with inhabitants >10000',
       'account_district no. of cities',
       'account_district ratio of urban inhabitants',
       'account_district average salary',
       'account_district unemploymant rate '95',
       'account_district unemploymant rate '96',
       'account_district unemploymant_growth',
       'account_district no. of enterpreneurs per 1000 inhabitants',
       'account_district no. of commited crimes '95',
       'account_district no. of commited crimes '96',
       'account_district crime_growth', 'account_district total_crime', 'code',
       'client_district no. of inhabitants',
       'cl